## Load data

In [1]:
import copy as cp
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt

In [2]:
import os, itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import sys
try: # ip: 153
    lib_s = '/home/adminuser/public/libs/pytorch_examples/basic_networks'
    sys.path.append(lib_s)
    import generate_model as nets
except: # ip : 103
    lib_s = '/home/bwlee/work/codes/examples/basic_networks'
    sys.path.append(lib_s)
    import generate_model as nets

## Make input data
interest.csv has only closed price.   
On the other hand, zipline needs OLHCV(?) format for each index  
make dummy columns and put 'close' value to them  
make csv file for each index  

In [3]:
f_s0 = 'interest.csv'
df0 = pd.read_csv(f_s0, index_col='date')
df0 = df0.iloc[::-1] # 과거에서 현재로 정렬순서 변경

### Add more data

In [8]:
class Environ():
    def __init__(self):
        self.indices = ['trea3', 'trea5', 'trea10']
        #self.tgt_index = 'trea10'
        self.tgt_index = 'trea3'
        # input window size
        self.input_size = 15
        # prediction is made after algo.pred_period from the present
        self.pred_period = 5 
env = Environ() # setting values

In [95]:
#dff0 = df0[:200] # test
dff0 = df0
dff = dff0

In [66]:
diff_periods = [int(env.input_size/4*x) for x in [1, 2, 3]]
for diff_period in diff_periods:
    diff = dff0.diff(periods=diff_period)
    ii = diff_period
    columns = ['d%d_diff_trea3'%(ii), 'd%d_diff_trea5'%(ii), 'd%d_diff_trea10'%(ii)]
    diff.columns = columns
    dff = dff.merge(diff, how='inner', on='date')
dff['diff35'] = dff['trea3'] - dff['trea5']
dff['diff310'] = dff['trea3'] - dff['trea10']
dff['diff510'] = dff['trea5'] - dff['trea10']

dff = dff.dropna()

## prepare data without time confusion
Seperate two actions

In [322]:
# normalized input, norm val, tgt_date, tgt_val, tgt_diff_ratio, 
def push_data(env, data, mkt_data):
    """
    read past data and push them to mkt_data
    :param data: dataframe info from csv file
    :param mkt_data: dataframe to be used for learning
    """
    #print(data)
    #print('----------')
    for ii_days in range(len(data)):
        if ii_days+1 < env.input_size:
            continue
        elif ii_days+1 == env.input_size:
            env.start_date = data.iloc[ii_days].name
        
        predicting_date = data.iloc[ii_days].name
        # historical data from today to env.input_size behind days
        history = data.iloc[ii_days+1-env.input_size:ii_days+1]
                
        ## normalize : look back 1 year
        lookback_step = 250
        lookback_step = env.input_size
        if ii_days+1 < lookback_step:
            cumul = data.iloc[:ii_days+1]
        else:
            cumul = data.iloc[ii_days+1-lookback_step:ii_days+1]
        """
        mean_val = cumul.mean() 
        range_val = 2*cumul.std()
        """
        mean_val = history.mean() 
        #range_val = history.max() - history.min()
        range_val = 2*history.std()
        norm_history = (history-mean_val)/range_val
        
        mkt_data[predicting_date] = [norm_history, mean_val, range_val, 
                                     None, None, None, None, None]
            
def push_tgt(env, data, mkt_data):
    """
    read future data and push them to mkt_data
    future_prediction_date is a future date when model want to predict
    past_predicting_date is a past date when model made a prediction with given information
    :param data: dataframe info from csv file
    :param mkt_data: dataframe to be used for learning
    """
    for ii_days in range(len(data)):
        ii_predicting = ii_days - env.pred_period
        predicting_date = data.iloc[ii_predicting].name
        if ii_predicting < 0 or predicting_date < env.start_date:
            continue
        
        past_history = mkt_data[predicting_date][0]
        mean_val = mkt_data[predicting_date][1]
        range_val = mkt_data[predicting_date][2]
        
        past_price = (past_history.iloc[-1]*range_val+mean_val)[env.tgt_index] # 49.6
        
        #past_price = (past_history.iloc[-5]*range_val+mean_val)[env.tgt_index] # 56.3
        
        #past_price = (past_history.iloc[-3]*range_val+mean_val)[env.tgt_index] # 53.0, # 53.0
        #past_price = (past_history.iloc[-13]*range_val+mean_val)[env.tgt_index] # 67.4
        #past_price = (past_history*range_val+mean_val).mean()[env.tgt_index] # 58.4
        #past_price = (past_history*range_val+mean_val)[:-3].mean()[env.tgt_index] # 63.2
        #past_price = (past_history*range_val+mean_val)[:-5].mean()[env.tgt_index] # 63.7
        
        #past_price = (past_history.iloc[-10]*range_val+mean_val)[env.tgt_index] # 66.4

        #past_price = (past_history.iloc[-5:].mean()*range_val+mean_val)[env.tgt_index] # 48.1
        #past_price = (past_history.iloc[-3:].mean()*range_val+mean_val)[env.tgt_index] # 48.1
        
        #past_price = (past_history.iloc[-10:].mean()*range_val+mean_val)[env.tgt_index] # 55.9
        #past_price = (past_history.iloc[-10:-5].mean()*range_val+mean_val)[env.tgt_index] # 60.4
        
        
        prediction_date = data.iloc[ii_days].name
        tgt_price = data.iloc[ii_days][env.tgt_index]
        #tgt_price = (data.iloc[ii_days-1:ii_days+2].mean())[env.tgt_index]
        
        tgt_ratio = (tgt_price - past_price)/past_price*10
        
        #mkt_data[predicting_date][0] = past_history[:-4]
        mkt_data[predicting_date][3] = prediction_date
        mkt_data[predicting_date][4] = np.single(tgt_price)
        mkt_data[predicting_date][5] = np.single(tgt_ratio)
        
        mkt_data[predicting_date][6] = np.single(past_price)
        mkt_data[predicting_date][7] = predicting_date

### Push past data

In [323]:
mkt_data = {} # object to store data for learning
push_data(env, dff, mkt_data)

### Check data are pushed correctly
* no dates should be ahead of key date
* prediction info are not pushed yet

In [156]:
keys = list(mkt_data.keys())[:30]
print(keys)

['2000-11-06', '2000-11-07', '2000-11-08', '2000-11-09', '2000-11-10', '2000-11-11', '2000-11-13', '2000-11-14', '2000-11-15', '2000-11-16', '2000-11-17', '2000-11-18', '2000-11-20', '2000-11-21', '2000-11-22', '2000-11-23', '2000-11-24', '2000-11-25', '2000-11-27', '2000-11-28', '2000-11-29', '2000-11-30', '2000-12-01', '2000-12-02', '2000-12-04', '2000-12-05', '2000-12-06', '2000-12-07', '2000-12-08', '2000-12-09']


In [97]:
mkt_data[keys[0]]

[               trea3     trea5    trea10
 date                                    
 2000-11-02  0.234349  0.241046  0.162245
 2000-11-03  0.269018  0.281901  0.192758
 2000-11-04  0.269018  0.281901  0.192758
 2000-11-06  0.303687  0.322756  0.230898
 2000-11-07  0.303687  0.330927  0.230898
 2000-11-08  0.173678  0.183849  0.108848
 2000-11-09 -0.329022 -0.363612 -0.364098
 2000-11-10 -0.727715 -0.698625 -0.623455
 2000-11-11 -0.727715 -0.698625 -0.623455
 2000-11-13 -0.831722 -0.804848 -0.760762
 2000-11-14 -0.797053 -0.813019 -0.760762
 2000-11-15 -0.641043 -0.674111 -0.646339
 2000-11-16 -0.563038 -0.592401 -0.608199
 2000-11-17 -0.589039 -0.616914 -0.631083
 2000-11-18 -0.589039 -0.616914 -0.631083,
 trea3     7.614808
 trea5     7.802500
 trea10    8.103654
 dtype: float64,
 trea3     0.576885
 trea5     0.611917
 trea10    0.655467
 dtype: float64,
 None,
 None,
 None,
 None,
 None]

In [50]:
env.start_date

'2000-11-18'

### Push future data

In [157]:
push_tgt(env, dff, mkt_data)
mkt_data = {key: val for key, val in mkt_data.items() if (val[-1] is not None)}

### CHECK tgt push is correctly made
end value of '2000-11-23' * norm == prediction value 

In [209]:
#print(mkt_data['2000-11-18'])
print(mkt_data['2000-11-11'])

[               trea3     trea5    trea10
date                                    
2000-10-26  0.300000  0.272099  0.293963
2000-10-27  0.105970  0.131358  0.152231
2000-10-28  0.105970  0.131358  0.152231
2000-10-30  0.091045  0.123951  0.152231
2000-10-31  0.150746  0.175802  0.183727
2000-11-01  0.105970  0.123951  0.136483
2000-11-02  0.128358  0.123951  0.104987
2000-11-03  0.158209  0.160988  0.136483
2000-11-04  0.158209  0.160988  0.136483
2000-11-06  0.188060  0.198025  0.175853
2000-11-07  0.188060  0.205432  0.175853
2000-11-08  0.076119  0.072099  0.049869
2000-11-09 -0.356716 -0.424198 -0.438320
2000-11-10 -0.700000 -0.727901 -0.706037
2000-11-11 -0.700000 -0.727901 -0.706037, trea3     7.664000
trea5     7.866333
trea10    8.143333
dtype: float64, trea3     0.670
trea5     0.675
trea10    0.635
dtype: float64, '2000-11-17', 7.275, 0.11118832, 7.195, '2000-11-11']


In [1]:
## Training
### Data preparation

In [136]:
class Index_data(Dataset):
    def __init__(self, in_data0, dates, addi=None):
        """
        :param in_data: dict of (df_past_normed, norm_val, prediction_date, tgt_price, tgt_ratio)
        :param addi: (optional) additional non-sequential elements that can be added to in_data
        """
        in_data = cp.deepcopy(in_data0)
        if addi is not None:
            assert list(in_data.keys()) == list(addi.keys())
             
        # change dataframe to float numpy array
        for key, val in in_data.items():
            # transform in_data into Tensor
            if not 'Tensor' in str(type(in_data[key][0])) :
                in_data[key][0] = torch.tensor(in_data[key][0].to_numpy(np.single))
                in_data[key][1] = torch.tensor(in_data[key][1].to_numpy(np.single))
                in_data[key][2] = torch.tensor(in_data[key][2].to_numpy(np.single))
            
            if addi is not None:
                in_data[key].append(addi[key])
            else:
                in_data[key].append('NULL')
            
        self.in_data = in_data
        self.i2dates = {}
        for ii, date1 in enumerate(dates):
             self.i2dates[ii] = date1
        
    def __getitem__(self, ii_date):
        """
        :return: (df_past_normed, norm_val, prediction_date, tgt_price, tgt_ratio)
        """
        predicting_date = self.i2dates[ii_date]
        return self.in_data[predicting_date]
    
    def __len__(self):
        return len(self.in_data)

In [158]:
#erase data that do not have prediction target
traintest_dates = sorted(list(mkt_data.keys()))
n_train = len(traintest_dates)*8//10
n_test = len(traintest_dates) - n_train
train_dates = traintest_dates[:n_train]
test_dates = traintest_dates[n_train:]

train_data = {}
test_data = {}
for key, val in mkt_data.items():
    if key in train_dates:
        train_data[key] = val
    elif key in test_dates:
        test_data[key] = val

In [159]:
## this is an example, I already add differences
def get_addi(in_data):
    #print(in_data)
    ret = {}
    for key, val in in_data.items():
        if not 'Tensor' in str(type(in_data[key][0])) :
            history_val = torch.tensor(val[0][['trea3','trea5', 'trea10']].to_numpy(np.single))
        else:
            history_val = val[0][:,:3] # only use three elements : ['trea3','trea5', 'trea10']
        addi1 = history_val[-1] - history_val[-5]
        addi2 = history_val[-1] - history_val[-7]
        addi3 = history_val[-1] - history_val[-10]
        addi4 = history_val[-1] - history_val[-3]
        addi5 = history_val[-3] - history_val[-7]
        addi6 = history_val[-3] - history_val[-10]
        addi7 = history_val[-5] - history_val[-7]
        addi8 = history_val[-5] - history_val[-10]
        addi = torch.cat((addi1, addi2, addi3, addi4, addi5, addi6, addi7, addi8), dim=-1)    
        ret[key] = addi
    return ret

In [160]:
batch_size = 64
#batch_size = 512

#train_dset = Index_data(train_data, train_dates)
#test_dset = Index_data(test_data, test_dates)
train_dset = Index_data(train_data, train_dates, get_addi(train_data))
test_dset = Index_data(test_data, test_dates, get_addi(test_data))

train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=batch_size, shuffle=False)

In [334]:
class Predictor_rnn(nets.Net):
    def __init__(self, rnn=None, downnet=None,
                 loss=None, optimizer=None, device='cuda:2',
                 dim_input=100, dim_hidden=200, bidirectional=True):
        """
        net is consists of [embed, rnn, downnet]
        :param downnet: define downstream job
        """
        super(Predictor_rnn, self).__init__(loss=loss, device=device)
        
        if rnn is None:
            self.rnn = nn.LSTM(input_size=dim_input, 
                               hidden_size=dim_hidden, 
                               num_layers= 2,
                               batch_first=True,
                               #bidirectional=False)
                               bidirectional=bidirectional)
        else:
            self.rnn = rnn
            
        if downnet is None:
            self.downnet = nets.get_MLP([dim_hidden, dim_hidden*2, 1], 
                                        dropout=0.35, end=True)
        else:
            self.downnet = downnet
        
        self.init_weights()
        
        self.rnn = self.rnn.to(device)
        self.downnet = self.downnet.to(device)
        
        parms = list(self.rnn.parameters())
        parms += list(self.downnet.parameters())
        self.optimizer = optimizer(parms)
        
    def set_train(self):
        self.rnn.train()
        self.downnet.train()
        
    def set_eval(self):
        self.rnn.eval()
        self.downnet.eval()
    
    def init_weights(self):
        for ii, layer in enumerate(self.downnet):
            if 'Linear' in str(layer):
                torch.nn.init.xavier_uniform_(self.downnet[ii].weight)
                #torch.nn.init.xavier_normal_(self.downnet[ii].weight)

    def forward(self, x, addi=('NULL')):
        out, hidden = self.rnn(x)
        out = out[:,-1] # choose last output
        if 'Tensor' in str(type(addi)):
            out = torch.cat((out, addi),dim=1)
        
        out = self.downnet(out) 
        return out.view(-1)

    def run_eval(self, data):
        self.set_eval()
        loss = 0
        outs = None
        tgts = None
        with torch.no_grad():
            for i_batch, data_batch in enumerate(data):
                data_in = data_batch[0]
                mean_val = data_batch[1]
                range_val = data_batch[2]
                pred_date = data_batch[3]
                tgt = data_batch[5]
                addi = data_batch[-1]
                
                data_in = data_in.to(self.device)
                tgt = tgt.to(self.device)
                if 'Tensor' in str(type(addi)):
                    addi = addi.to(self.device)
                    
                out = self.forward(data_in, addi)
                #print('loss', loss, self.loss)
                loss += self.loss(out, tgt).cpu().numpy()
                out = out.cpu().numpy()
                #print('loss===', loss)
                tgt = tgt.cpu().numpy()
                if outs is None:
                    outs = out
                    tgts = tgt
                else:
                    outs = np.concatenate((outs, out), axis=0)
                    tgts = np.concatenate((tgts, tgt), axis=0)
        loss /= 1.0*(i_batch+1)
        print('evaluate', 'loss', loss, 'accuracy : define function')
        return outs, tgts, loss
    
    def run_batch(self, i_batch, data_batch):
        self.optimizer.zero_grad()
        data_in = data_batch[0]
        mean_val = data_batch[1]
        range_val = data_batch[2]
        pred_date = data_batch[3]
        tgt = data_batch[5]
        addi = data_batch[-1]
        
        data_in = data_in.to(self.device)
        tgt = tgt.to(self.device)
        if 'Tensor' in str(type(addi)):
            addi = addi.to(self.device)
        out = self.forward(data_in, addi)
        
        loss = self.loss(out, tgt)
        loss.backward()
        self.optimizer.step()
        return loss.detach().cpu().item()

In [335]:
dim_input = train_dset[0][0].shape[-1]
dim_hidden = 400 
#downnet = nets.get_MLP([dim_hidden+3*3, dim_hidden, 1], 

# Add additional variables
# no additional
dim_hiddenb = dim_hidden 
# add 8*3(above examples) variables
dim_hiddenb = dim_hidden+3*8 

downnet = nets.get_MLP([dim_hiddenb, dim_hidden, 1], 
                        dropout=0.35, end=True)
#bidownnet = nets.get_MLP([dim_hidden*2+3*8, dim_hidden*2, 1], 
#                        dropout=0.0, end=True)
loss = nn.MSELoss() # which combines logsoftmax and nll loss
optimizer = optim.Adam
device = torch.device("cuda:2")

In [336]:
predictor = Predictor_rnn(loss=loss, optimizer=optimizer, 
                          device=device, dim_input=dim_input, 
                          dim_hidden=dim_hidden, downnet=downnet, bidirectional=False)
                          #dim_hidden=dim_hidden, downnet=bidownnet, bidirectional=True)

In [337]:
predictor.run_train(n_epoch=30, data=train_loader, test_data=test_loader)

epoch 0 loss 0.10564924726512895
evaluate loss 0.13288307430989602 accuracy : define function
epoch 1 loss 0.10100839070197362
evaluate loss 0.13254390350159476 accuracy : define function
epoch 2 loss 0.09706368703228324
evaluate loss 0.13254819525515332 accuracy : define function
epoch 3 loss 0.09596463087112156
evaluate loss 0.13333280296886668 accuracy : define function
epoch 4 loss 0.09400750490934101
evaluate loss 0.13271797733271823 accuracy : define function
epoch 5 loss 0.09401697315164466
evaluate loss 0.13312692195177078 accuracy : define function
epoch 6 loss 0.09302833081403775
evaluate loss 0.13196822803686648 accuracy : define function
epoch 7 loss 0.09344892871024003
evaluate loss 0.13216696701505604 accuracy : define function
epoch 8 loss 0.09334946757377084
evaluate loss 0.13126041148515308 accuracy : define function
epoch 9 loss 0.09316687158017016
evaluate loss 0.13249778090154424 accuracy : define function
epoch 10 loss 0.0935307571025037
evaluate loss 0.13245337828

In [338]:
rets = {}
tgts = None
preds = None
predictor.set_eval()
with torch.no_grad():
    for data_batch in test_loader:
        data_in = data_batch[0]
        tgt = data_batch[5]
        addi = data_batch[-1]
        
        data_in = data_in.to(device)
        if 'Tensor' in str(type(addi)):
            addi = addi.to(device)
            
        tgt = tgt.numpy()
        out = predictor.forward(data_in, addi).cpu().numpy()
        if preds is None:
            preds = out
            tgts = tgt
        else:
            preds = np.concatenate((preds, out), axis=0)
            tgts = np.concatenate((tgts, tgt), axis=0)
            
        prediction_dates = data_batch[3]
        past_prices = data_batch[6]
        predicting_dates = data_batch[7]
        for ii, date1 in enumerate(prediction_dates):
            out1 = out[ii]
            out_orig = out1/10*past_prices[ii] + past_prices[ii]
            rets[date1] = {'predicting_date': predicting_dates[ii],
                          'pred': out_orig,
                           'past': past_prices[ii],
                           'tgt': data_batch[4][ii]}
            
preds = preds.reshape(-1)
tgts = tgts.reshape(-1)

In [339]:
list(rets.keys()).index('2021-01-04')

999

In [340]:
limit = 999
sum1 = np.sum((preds[limit:] > 0) & (tgts[limit:] > 0))
sum2 = np.sum((preds[limit:] < 0) & (tgts[limit:] < 0))
print(sum1, np.sum(tgts[limit:]>0))
print(sum2, np.sum(tgts[limit:]<0))
print(len(preds[limit:]), sum1+sum2, 1.0*(sum1+sum2)/len(preds[limit:]))

28 44
8 24
71 36 0.5070422535211268


In [341]:
sum1 = np.sum((preds > 0) & (tgts > 0))
sum2 = np.sum((preds < 0) & (tgts < 0))
print(sum1, np.sum(tgts>0))
print(sum2, np.sum(tgts<0))
print(len(preds), sum1+sum2, 1.0*(sum1+sum2)/len(preds))

272 482
313 570
1070 585 0.5467289719626168


## all includes 3*3=9 diff, predict 4 days avg, prediction_period=10

lstm_layer=2  
epoch 30, dropout=0, dim_hidden=100, mlp (dim_hidden, dim_hidden, 1) --> 55.5%  

epoch 30, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, 1) --> 55.3%  

epoch 30, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden*2, 1) --> 53.4%  


epoch 30, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden//2, 1) --> 53.4%  
epoch 20, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden//2, 1) --> 53.9%  

epoch 20, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, dim_hidden, 1) --> 48.8%  
epoch 30, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, dim_hidden, 1) --> 51.4%  
epoch 40, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, dim_hidden, 1) --> 53.7%  
epoch 50, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, dim_hidden, 1) --> 49.1%  
epoch 60, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, dim_hidden, 1) --> 48.7%  
epoch 70, dropout=0, dim_hidden=200, mlp (dim_hidden, dim_hidden, dim_hidden, 1) --> 50.3%  


epoch 10, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 52.2%  
epoch 20, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 52.0%  
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 56.9%, 54.3%  
epoch 40, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.9%  
epoch 50, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 53.3%  
epoch 60, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 48.6%, 49.6  

epoch 30, dropout=0, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.2  
epoch 40, dropout=0, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.8  
epoch 50, dropout=0, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 49.8  

epoch 30, dropout=0, dim_hidden=500, mlp (dim_hidden, dim_hidden, 1) --> 55.8  
epoch 40, dropout=0, dim_hidden=500, mlp (dim_hidden, dim_hidden, 1) --> 53.8  

lstm_layer=1
epoch 20, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 52.3%  
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.7%  

lstm_layer=3
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 55.7%  
epoch 40, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 55.4%  
epoch 50, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 53.8%  

## includes 3*8=24 diff
layer=2
epoch 20, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 52.5%  
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.3%  
epoch 40, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 53.9%  

## includes 3*8=24 diff, 1day prediction
layer=2
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 53.8%  

Below prediction days are (-2,-1,0)
## includes 3*8=24 diff, 3day prediction
layer=2
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 53.7%  

## includes 3*8=24 diff, 3day predicting, 3day prediction
layer=2
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 58.5%  

## includes 3*8=24 diff, 3day predicting, 3day prediction, prediction_period=5
layer=2
epoch 10, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 58.8%  
epoch 20, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 62.6%  
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 58.5%  
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 58.3%  

## includes 3*8=24 diff, 3day predicting, 3day prediction, prediction_period=15
layer=2
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.3%  

## includes 3*8=24 diff, 3day predicting, 3day prediction, prediction_period=20
layer=2
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.7%  
epoch 40, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 55.1%  
epoch 50, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 55.1%  
epoch 60, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 51.4%  

##  includes 3*8=24 diff, last price, 3day prediction, layer=2
epoch 5, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 51.7%  
epoch 10, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 56.4%, 54.8, 54.6  
epoch 10, dropout=0.35, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 52.7  
epoch 20, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 55.5%, 53.9 
epoch 20, dropout=0.35, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.3
epoch 30, dropout=0, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 49.3% 
epoch 30, dropout=0.35, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 55.2% 
epoch 40, dropout=0.35, dim_hidden=300, mlp (dim_hidden, dim_hidden, 1) --> 54.5% 

epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.3  
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.5  
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.3  
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.3  
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 53.9  

## Bidirectional
epoch 10, dropout=0., dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 55.9 
epoch 20, dropout=0., dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 55.2 
epoch 30, dropout=0., dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 52.5 
epoch 40, dropout=0., dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 52.5 
epoch 50, dropout=0., dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 52.8 

epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 56.7  
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 54.1 
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 55.1 
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 56.4 
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 55.0 
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden*2, dim_hidden*2, 1) --> 54.6 

## last to last period 5
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 53.9
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.4
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.0
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.2
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.1
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 53.9

## last to last period 10
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 50.0
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.6
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 58.1
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.3
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.5
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 49.1

## last to last period 15
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 50.3, 51.6
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.2, 52.8
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.8, 54.5
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.2, 52.1
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 53.5, 50.7
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.3, 52.2
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 51.1, 48.4

## last to last period 20
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.6
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 50.7
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.1
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 51.5
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 48.6
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 47.0

## last to pred3 period 5
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 47.0  
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.1  
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.0  
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.4  
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.4  
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.3  
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.5  
epoch 80, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.2  
epoch 90, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.6  

## last to pred3 period 10
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.6
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.2
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.6
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 58.4
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.5
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.1
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 53.7
epoch 80, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 51.4

## last to pred3 period 15
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.0
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.9
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.0
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.0
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 51.6
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 49.3

## last to pred3 period 20
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 50.4
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.8
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.4
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 51.5
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 49.3
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 51.5
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 50.1

## pred3 to last period 5
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 61.0  
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 60.1  
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 61.9  
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 60.5  
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 61.0  
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 59.5  
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 59.3  
epoch 80, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.6  

## pred3 to last period 10
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 59.0
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 59.1
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 59.8
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 58.6
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 58.7
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.9
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 57.5
epoch 80, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.1

## pred3 to last period 15
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.9
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.9
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.6
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.0
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.9
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 53.3
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 50.5

## pred3 to last    period 20
epoch 10, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 52.6
epoch 20, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.4
epoch 30, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 56.4
epoch 40, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.8
epoch 50, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 54.1
epoch 60, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 55.2
epoch 70, dropout=0.35, dim_hidden=400, mlp (dim_hidden, dim_hidden, 1) --> 49.2

In [22]:
rets_2021 = {key: val for key, val in rets.items() if key>'2021-01-01'}
rows = list(rets_2021['2021-01-04'].keys())
cols = list(rets_2021.keys())

In [27]:
rets_2021b = {}
for col in cols:
    for row in rows:
        rets_2021b.setdefault(row, {})
        val = rets_2021[col][row]
        if type(val) is not str:
            rets_2021b[row][col] = rets_2021[col][row].item()        
        else:
            rets_2021b[row][col] = rets_2021[col][row]

In [28]:
pd.DataFrame(rets_2021b).to_csv('./rets.csv', index=True)

In [21]:
rets_2021

{'2021-01-04': {'predicting_date': '2020-12-24',
  'pred': tensor(0.9607),
  'past': tensor(0.9403),
  'tgt': tensor(0.9550)},
 '2021-01-05': {'predicting_date': '2020-12-28',
  'pred': tensor(0.9542),
  'past': tensor(0.9453),
  'tgt': tensor(0.9370)},
 '2021-01-06': {'predicting_date': '2020-12-29',
  'pred': tensor(0.9781),
  'past': tensor(0.9630),
  'tgt': tensor(0.9600)},
 '2021-01-07': {'predicting_date': '2020-12-30',
  'pred': tensor(0.9640),
  'past': tensor(0.9690),
  'tgt': tensor(0.9700)},
 '2021-01-08': {'predicting_date': '2020-12-31',
  'pred': tensor(0.9614),
  'past': tensor(0.9733),
  'tgt': tensor(0.9770)},
 '2021-01-11': {'predicting_date': '2021-01-04',
  'pred': tensor(0.9480),
  'past': tensor(0.9650),
  'tgt': tensor(0.9650)},
 '2021-01-12': {'predicting_date': '2021-01-05',
  'pred': tensor(0.9365),
  'past': tensor(0.9540),
  'tgt': tensor(0.9770)},
 '2021-01-13': {'predicting_date': '2021-01-06',
  'pred': tensor(0.9564),
  'past': tensor(0.9507),
  'tgt': t